In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 1s 1us/step


In [3]:
# Membuka dan membaca file yang ditentukan.
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# Menghitung dan mencetak jumlah karakter unik dalam teks.
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars # Menghasilkan output dari operasi sebelumnya.

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
# Mendefinisikan lapisan StringLookup.
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
# Mengonversi karakter-karakter Unicode menjadi ID numerik.
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
# Membuat lapisan StringLookup baru.
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy() # Menggabungkan karakter-karakter Unicode dalam chars.

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1) # Mengonversi ID numerik kembali menjadi karakter.

In [13]:
# Ambil tensor yang berisi kode unicode untuk bagi string 'text' jadi unicode.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # Buat dataset dari all_ids.

In [15]:
for ids in ids_dataset.take(10): # loop untuk ambil 10 elemen pertama dataset ids_dataset.
  print(chars_from_ids(ids).numpy().decode('utf-8')) # Ambil elemen ids.

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100 # Jumlah sequence.

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # Jika panjang ID tidak habis dibagi oleh seq_length + 1, maka elemen yang tersisa akan dihapus.

for seq in sequences.take(1): # Ambil satu sekuenes /batch.
  print(chars_from_ids(seq)) # Cetak karakter.

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5): # Ambil 5 sekuens pertama.
  print(text_from_ids(seq).numpy()) # Cetak teks.

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence):
  input_text = sequence[:-1] # Ambil semua elemen kecuali elemen terakhir dai sekuens.
  target_text = sequence[1:] # Ambil semua elmeen kecuali elemen pertama dari sekuens.
  return input_text, target_text # Mengembalikan nilai.

In [20]:
split_input_target(list("Tensorflow")) # Bagi sequence jadi 2.

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target) # Mapping sekuens dalam dataset ke pasangan input dan target.

In [22]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy()) # Cetak label input.
  print("Target:", text_from_ids(target_example).numpy()) # Cetak label target.

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE) # Acak elemen.
    .batch(BATCH_SIZE, drop_remainder=True) # Mengelompokkan elemen dataset menjadi batch.
    .prefetch(tf.data.experimental.AUTOTUNE)) # Mengoptimalkan proses memuat data.

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256 # Jumlah embedding untuk represntasi vektor karakter.

# Number of RNN units
rnn_units = 1024 # Jumlah unit dalam RNN.

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states # Mengembalikan urutan karakter yang diprediksi serta status terakhir RNN.
    else:
      return x # Hanya akan mengembalikan urutan karakter yang diprediksi.

In [26]:
model = MyModel(
    vocab_size=vocab_size, # Menentukan seberapa banyak karakter yang ada dalam vokabulari yang digunakan oleh model.
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch) # Membuat prediksi dengan memberikan input_example_batch sebagai input.
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary() # Cetak ringkasan (summary) dari model TensorFlow.

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy() # Menghapus dimensi tambahan dan menghasilkan tensor satu dimensi.

In [30]:
sampled_indices # Indeks karakter yang diambil secara acak dari prediksi model.

array([40, 41, 54, 18, 58, 65, 24, 43, 59, 14, 37, 63,  2,  8, 54, 41, 34,
       56, 33, 33, 37, 33, 40, 10, 31, 56, 40,  8, 62, 24, 58, 65, 51, 57,
       57, 54, 40, 56, 31,  3,  4, 46, 57, 24,  5, 33, 59, 42, 60, 41, 30,
       27,  0, 51, 31, 37, 23, 36, 30, 38,  6, 11, 15,  7, 41, 25, 33,  0,
       43,  8, 54, 61,  8, 12,  0, 24, 37, 14,  2, 27, 63, 46,  6, 38, 19,
       14, 14, 34,  7,  6, 60, 37, 24, 55,  8, 39, 20, 47, 12, 36])

In [31]:
# Mencetak teks dari input dan prediksi karakter selanjutnya dari model.
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"eping, weeping and blubbering.\nStand up, stand up; stand, and you be a man:\nFor Juliet's sake, for h"

Next Char Predictions:
 b"aboEszKdtAXx -obUqTTXTa3Rqa-wKszlrroaqR!$grK&TtcubQN[UNK]lRXJWQY':B,bLT[UNK]d-ov-;[UNK]KXA Nxg'YFAAU,'uXKp-ZGh;W"


In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) # Menginisialisasi fungsi kerugian.

In [33]:
# Menghitung dan mencetak informasi tentang hasil prediksi dan kerugian.
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1897993, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy() # Menghasilkan nilai eksponensial dari example_batch_mean_loss.

66.009544

In [35]:
model.compile(optimizer='adam', loss=loss) # Mengatur pengaturan pelatihan untuk model.

In [36]:
# Menentukan cara untuk menyimpan checkpoint selama pelatihan model.
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
EPOCHS = 20

In [38]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback]) # Melatih model menggunakan data dari dataset.

Epoch 1/20
172/172 [==============================] - 1000s 6s/step - loss: 2.7432
Epoch 2/20
172/172 [==============================] - 999s 6s/step - loss: 2.0058
Epoch 3/20
172/172 [==============================] - 1002s 6s/step - loss: 1.7205
Epoch 4/20
172/172 [==============================] - 998s 6s/step - loss: 1.5557
Epoch 5/20
172/172 [==============================] - 999s 6s/step - loss: 1.4557
Epoch 6/20
172/172 [==============================] - 1021s 6s/step - loss: 1.3864
Epoch 7/20
172/172 [==============================] - 1012s 6s/step - loss: 1.3335
Epoch 8/20
172/172 [==============================] - 1010s 6s/step - loss: 1.2896
Epoch 9/20
172/172 [==============================] - 1006s 6s/step - loss: 1.2486
Epoch 10/20
172/172 [==============================] - 1004s 6s/step - loss: 1.2096
Epoch 11/20
172/172 [==============================] - 1002s 6s/step - loss: 1.1712
Epoch 12/20
172/172 [==============================] - 1001s 6s/step - loss: 1.1308
Epoc

In [39]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [40]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
# Menghasilkan teks berkelanjutan berdasarkan teks awal "ROMEO:".
start = time.time() # Memulai penghitungan waktu.
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

# Loop sebanyak 1000 kali.
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# Cetak bersama dengan waktu yang diperlukan.
result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
You say, he says he indeed.

LORD WILLOUGHBY:
Bareless, my great boy.

Provost:
As 'twere a son of King of Elyer's life.
What shall scarce be so so budded in your bed
And swell proclaim our Romeo calm,
Could not less furtle villain: let me hear from help.

KING HENRY VI:
My Lord of Warwick, hear me breaking me married
Jaggage-men, to dance his leap: you make him wink
Help not all framed away their fines. Putticotion may soon
Rest; these nothing blest musicians endem:
If I thought made Glunterence of the nurse,
Not set up all then. Your from his baggacies
He could draw it in the witness of his
mistress'd Paulina.
Ancies spriad of a king, great king, we fear
There is a fire show'd upon me.

Third Watchman:
A marb, stone, of whom defend a word-well incring
As else proud and these fat as civelity,
I mean to blass along where you'll strive, let
To chase your only long battle; if I
Hold in post, and are consented
He was a penural delighters,
Which plotted most growing thus have great,

In [42]:
# Menghasilkan teks berkelanjutan berdasarkan teks awal yang telah ditentukan.
start = time.time() # Memulai penghitungan waktu.
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

# Loop sebanyak 1000 kali.
for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

# Cetak bersama dengan waktu yang diperlukan.
result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nCanst thou yet?\n\nClown:\nCousin, I pardon him, as we, as bright-late\nCarble all your enemies, lives, to art,\nWho from pay thee from thy face the joy may\nplease you, lets' cut off blush and blunt,\nIf ever yet let me be vail'd before to be\nbrief; we sathon dischined at barbard he of while.\n\nDUKE OF YORK:\nShe may do mine sorrow, think ut you may stay.\n\nHENRY BOLINGBROKE:\nYou shames nothing but shortly for it, it is no daughter.\nGood paint before thy way so well as I do,--\nIf you have set a woman and amends;\nUncle, Signior Gremio, how farewell gown.\nMy fair Bohemia, how fon your grace I meant,\nMy sovereign royal marge my sons, we\nchose this crown and cover.\nCome on, by Gaint.\n\nEDBAL:\nAlack, protector: I have from your voices and\nThe villain's soothing news indirmponisgman:\nYour brother Richard, noing of this face\nWhere you should not believe I since you do excuse my head\nAnd prone with the remembrance of a great divinu.\nWell met, for I inte

In [43]:
# Menyimpan model one_step_model ke dalam format saved model.
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [44]:
# Menggunakan model yang telah dilatih untuk menghasilkan teks berkelanjutan.
states = None # Menyimpan keadaan internal dari model.
next_char = tf.constant(['ROMEO:']) # Teks awal untuk menghasilkan teks berikutnya.
result = [next_char]

# Loop sebanyak 100 kali.
for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8")) # Cetak hasil akhir teks.

ROMEO:
His name is Came, 'Citizens! make all medly player
To your succersise whence thou office; off but n


**Tugas**

In [45]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [46]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [47]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [48]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 1030s 6s/step - loss: 2.7058


In [49]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])

    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

      # saving (checkpoint) the model every 5 epochs
      if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

        print()
        print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
        print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
        print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1959
Epoch 1 Batch 50 Loss 2.0346
Epoch 1 Batch 100 Loss 1.9226
Epoch 1 Batch 150 Loss 1.8578
Epoch 2 Batch 0 Loss 1.7885
Epoch 2 Batch 50 Loss 1.7323
Epoch 2 Batch 100 Loss 1.6744
Epoch 2 Batch 150 Loss 1.6434
Epoch 3 Batch 0 Loss 1.5567
Epoch 3 Batch 50 Loss 1.5859
Epoch 3 Batch 100 Loss 1.6036
Epoch 3 Batch 150 Loss 1.4830
Epoch 4 Batch 0 Loss 1.4267
Epoch 4 Batch 50 Loss 1.3942
Epoch 4 Batch 100 Loss 1.4475
Epoch 4 Batch 150 Loss 1.4286
Epoch 5 Batch 0 Loss 1.3921

Epoch 5 Loss: 1.3921
Time taken for 1 epoch 9.00 sec
________________________________________________________________________________
Epoch 5 Batch 50 Loss 1.3536

Epoch 5 Loss: 1.3842
Time taken for 1 epoch 297.73 sec
________________________________________________________________________________
Epoch 5 Batch 100 Loss 1.4340

Epoch 5 Loss: 1.3819
Time taken for 1 epoch 587.35 sec
________________________________________________________________________________
Epoch 5 Batch 150 Loss 1.3628

Epoch

Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

* Pada praktikum 2 menggunakan Model Fit sedangkan pada kode diatas menggunakan Model Teacher-Forcing dimana model ini merupakan pendekatan pelatihan yang melibatkan pemberian jawaban yang benar sebagai input selama proses pelatihan. Ini lebih spesifik terkait dengan cara pelatihan model untuk tugas tertentu, seperti generasi urutan.